# Algoritmo de Inferencia por Verificacion de Modelos

----

Anteriormente, ya hemos estudiado la estructura general de un agente lógico y como
podemos realizar la construcción del conocimiento a medida que el agente toma
diferentes percepciones en el espacio.

El objetivo de este problema es implementar la inferencia lógica del agente del mundo de
Wumpus con el fin de concluir que no hay un hoyo en la casilla [1,2]

### Conocimiento Base

Almacena las sentencias del agente, permite hacer queries de acciones o perceptos realizados

In [ ]:
class BaseKnowledge:
    def __init__(self):
        self.sentences = []

    def tell(self, sentence: str):
        """Stores a proposition in the knowledge base."""
        self.sentences.append(sentence)

    def ask(self, query: str) -> str | None:
        """Returns the most recent proposition that matches the query."""
        for sentence in reversed(self.sentences):
            if query in sentence:
                return sentence

    def __str__(self) -> str:
        return 'Sentences:\n' + '\n'.join(self.sentences)

### Funciones

`TT_Entails` es la funcion principal que ejecuta la verificacion

`TT_Check_All` Itera recursivamente sobre todos los simbolos para cubrir todas las combinaciones de la tabla de verdad de la proposicion

`PL_True_KB` Verifica que todas las proposiciones añadidas en el conocimiento base son verdaderas

`PL_True` Evalua la proposicion dada dado un modelo (el modelo es dado por TT_Check_All en cada iteracion), separando cada lado de cada operacion logica hasta que se evalue toda la proposicion

`extract_symbols` Extrae todos los simbolos presentes tanto en el conocimiento base como en la proposicion que se esta verificando

In [ ]:
def TT_Entails(KB, alpha):
    """Checks if KB entails alpha using model checking."""
    symbols = extract_symbols(KB, alpha)
    return TT_Check_All(KB, alpha, symbols, {})

def TT_Check_All(KB, alpha, symbols, model):
    if not symbols:
        if PL_True_KB(KB, model):
            return PL_True(alpha, model)
        else:
            return True
    else:
        P = symbols[0]
        rest = symbols[1:]
        return (TT_Check_All(KB, alpha, rest, {**model, P: True}) and
                TT_Check_All(KB, alpha, rest, {**model, P: False}))

def PL_True_KB(KB, model):
    """Evaluates if all propositions in the KB are true in a model."""
    for sentence in KB.sentences:
        if not PL_True(sentence, model):
            return False
    return True

def PL_True(sentence, model):
    """Evaluates if a proposition is true in a given model."""
    if '↔' in sentence:
        left, right = sentence.split('↔')
        return PL_True(left.strip(), model) == PL_True(right.strip(), model)
    elif '∨' in sentence:
        disjuncts = sentence.split('∨')
        return any(PL_True(d.strip(), model) for d in disjuncts)
    elif '∧' in sentence:
        conjuncts = sentence.split('∧')
        return all(PL_True(c.strip(), model) for c in conjuncts)
    elif '¬' in sentence:
        prop = sentence.replace('¬', '').strip()
        return not model.get(prop, False)
    else:
        return model.get(sentence.strip(), False)

def extract_symbols(KB, alpha):
    """Extracts the propositional symbols from the KB and the query."""
    symbols = set()
    for sentence in KB.sentences:
        symbols.update(sentence.replace('¬', '').replace('∧', '').replace('∨', '').split())
    symbols.update(alpha.replace('¬', '').replace('∧', '').replace('∨', '').split())
    return list(symbols)

### Main

In [ ]:
kb = BaseKnowledge()
kb.tell('¬P1,1')  # No Pit on (1,1)
kb.tell('B1,1 ↔ (P1,2 ∨ P2,1)')  # Breeze in (1,1) implies Pit en (1,2) o (2,1)
kb.tell('B2,1 ↔ (P1,1 ∨ P2,2 ∨ P3,1)')  # Breeze in (2,1) implies Pit in (1,1), (2,2) o (3,1)

# Update the KB with perceptions
percepts = ['¬B1,1', '¬B1,2']
for percept in percepts:
    kb.tell(percept)

# Update the KB with additional inferences
if '¬B1,1' in kb.sentences:
    kb.tell('¬P1,2')
    kb.tell('¬P2,1')

# Query if it's true that there's no Pit in (1,2)
query = '¬P1,2'
if TT_Entails(kb, query):
    print(f"Se infiere que {query} es verdadero.")
    kb.tell(query)
else:
    print(f"No se puede inferir que {query} sea verdadero.")
print(kb)